<a href="https://colab.research.google.com/github/giani-reyess/DataAnalysis_MAL/blob/main/MAL_DA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Dependencies**

In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns 
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
# !git clone https://github.com/MaxHalford/prince.git
# !pip install prince
import prince 

In [18]:
# Create df copy
df_anime = pd.read_csv('anime.csv', error_bad_lines=False,engine="python")
df_copy = df_anime.copy(deep=True)
df_copy = df_copy.dropna()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
Skipping line 3379: Expected 39 fields in line 3379, saw 44
Skipping line 4138: Expected 39 fields in line 4138, saw 44


#**Fixing dataset for preprocessing**

### General cleaning

In [19]:
# Slicing irrelevant data from dataset
df_copy = df_copy.iloc[0:,1:29]

# Drop irrelevant and repeated columns
df_copy.drop(['start_year', 'start_date', 'end_date', 'sfw'], axis=1, inplace=True)
df_copy.head(2)

,title,type,score,scored_by,status,episodes,source,members,favorites,episode_duration,...,updated_at,start_season,real_start_date,real_end_date,broadcast_day,broadcast_time,genres,themes,demographics,studios
2,Shingeki no Kyojin Season 3 Part 2,tv,9.07,1329500,finished_airing,10,manga,1881734.0,51931,0 days 00:23:55,...,2022-04-18 05:02:23+00:00,spring,2019-04-29,2019-07-01,monday,00:10:00,"['Action', 'Drama']","['Gore', 'Military', 'Survival']",['Shounen'],['Wit Studio']
3,Steins;Gate,tv,9.08,1252286,finished_airing,24,visual_novel,2269121.0,173088,0 days 00:24:20,...,2022-05-01 21:24:42+00:00,spring,2011-04-06,2011-09-14,wednesday,02:05:00,"['Drama', 'Sci-Fi', 'Suspense']","['Psychological', 'Time Travel']",[],['White Fox']


### 'episode_duration' cleaning

In [20]:
# Delete first 8 caracters for each value in from 'episode_duration' column 
df_copy['episode_duration'] = df_copy['episode_duration'].astype(str).apply(lambda x: x[8:])
df_copy['episode_duration']

2       0:23:55
3       0:24:20
10      0:23:00
11      0:23:55
12      0:22:55
         ...   
5638    0:23:00
5639    0:24:45
5641    0:23:40
5654    0:23:40
5710    0:23:00
Name: episode_duration, Length: 612, dtype: object

###Multicategorical variables cleaning

In [21]:
df_copy['demographics'].tail(2)

5654    ['Shounen']
5710             []
Name: demographics, dtype: object

In [26]:
# Convert from strings to arrays

# df_copy['genres'] = df_copy['genres'].astype(str).dropna().apply(eval)
# df_copy['themes'] = df_copy['themes'].astype(str).dropna().apply(eval)
# df_copy['demographics'] = df_copy['demographics'].astype(str).dropna().apply(eval)
# df_copy['studios'] = df_copy['studios'].astype(str).dropna().apply(eval)

# Convert array to string

df_copy['demographics'] = df_copy['demographics'].apply(lambda x: ''.join(x))
df_copy['studios'] = df_copy['studios'].apply(lambda x: ''.join(x))


df_copy['studios'].values
df_copy['demographics'].values


array(['Shounen', '', 'Shounen', 'Shounen', 'Shounen', 'Shounen', '',
       'Shounen', 'Shounen', 'Seinen', 'Shounen', '', 'Shounen',
       'Shounen', 'Shounen', 'Seinen', 'Shounen', '', 'Seinen', '', '',
       '', 'Shounen', '', '', '', '', '', '', '', '', '', 'Shounen',
       'Shoujo', 'Shounen', '', '', 'Seinen', '', '', '', '', 'Shoujo',
       'Shounen', 'Shounen', 'Shounen', '', '', '', 'Shounen', '', '', '',
       'Shounen', 'Shounen', '', '', '', '', 'Shounen', '', 'Seinen',
       'Shounen', '', 'Shounen', 'Seinen', 'Shounen', 'Shounen', 'Shoujo',
       'Shounen', 'Shoujo', '', 'Shounen', '', '', 'Josei', 'Shoujo', '',
       '', 'Shounen', '', '', '', '', 'Shounen', '', 'Shounen', 'Shounen',
       '', 'Josei', 'Shounen', 'Shounen', 'Shounen', '', '', '', '',
       'Seinen', 'Shounen', 'Seinen', 'Shounen', 'Shounen', '', '', '',
       '', 'Shounen', 'Josei', 'Seinen', '', 'Shounen', 'Shoujo',
       'Seinen', 'Shounen', 'Shounen', 'Josei', '', 'Seinen', '',
       'Sh

In [24]:
# Count unique values

def count_uniques(series):
 return pd.Series([j for i in series for j in i])

unique_genres = count_uniques(df_copy['genres']).value_counts()
unique_themes = count_uniques(df_copy['themes']).value_counts()

unique_genres
unique_themes

School              168
Harem                62
Mecha                53
Super Power          46
Military             45
Psychological        43
Isekai               42
Mythology            35
Music                29
Historical           26
Gag Humor            24
Adult Cast           23
Gore                 21
Vampire              20
Workplace            19
Parody               19
Love Polygon         19
Mahou Shoujo         15
Survival             15
Reincarnation        14
Space                14
Video Game           13
High Stakes Game     13
Iyashikei            13
Time Travel          12
CGDCT                12
Idols (Female)       12
Otaku Culture        11
Samurai              10
Martial Arts          9
Anthropomorphic       7
Team Sports           7
Romantic Subtext      6
Visual Arts           6
Performing Arts       5
Detective             5
Delinquents           4
Organized Crime       3
Crossdressing         2
Reverse Harem         1
Strategy Game         1
Childcare       

In [27]:
# Create a column for each genre where values will be boolean

def boolean_df(item_lists, unique_items):
# Create empty dict
    bool_dict = {}
    
    # Loop through all the tags
    for i, item in enumerate(unique_items):
        
        # Apply boolean mask
        bool_dict[item] = item_lists.apply(lambda x: item in x)
            
    # Return the results as a dataframe
    return pd.DataFrame(bool_dict) 

genre_bool = pd.DataFrame(boolean_df(df_copy['genres'], unique_genres.keys())).astype(int) 
themes_bool = pd.DataFrame(boolean_df(df_copy['themes'], unique_themes.keys())).astype(int)

themes_bool
genre_bool

,Action,Comedy,Fantasy,Romance,Sci-Fi,Adventure,Drama,Supernatural,Mystery,Slice of Life,Ecchi,Horror,Sports,Suspense,Gourmet,Avant Garde,Boys Love,Girls Love
2,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0
10,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0
11,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
12,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5638,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5639,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5641,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5654,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


#**Preprocessing**

###Categoriacal variables 

In [ ]:
# categ_var_encoded = pd.get_dummies(df_copy[['type','title','status','source', 'start_season', 'broadcast_day','real_start_date', 'real_end_date', 'broadcast_time']])
# categ_var_encoded

###Numerical variables

In [28]:
numerical_df = pd.DataFrame(df_copy[['score','scored_by','episodes','members','favorites']])
numerical_df_scaled = pd.DataFrame(StandardScaler().fit_transform(numerical_df), columns = ['score','scored_by','episodes','members','favorites'])
numerical_df_scaled

,score,scored_by,episodes,members,favorites
0,3.409509,3.320016,-0.420995,2.841992,2.771764
1,3.432088,3.078444,0.123471,3.632225,10.265577
2,2.393430,6.829660,0.629046,6.052196,9.297283
3,2.528907,3.069947,0.123471,2.924724,4.014620
4,2.483748,2.938774,0.045690,3.006447,4.499850
...,...,...,...,...,...
607,-1.512828,-0.694190,0.045690,-0.741772,-0.387764
608,-1.332192,-0.735072,-0.304324,-0.830818,-0.394258
609,-0.361272,-0.815796,-0.343214,-0.949975,-0.432421
610,-0.835443,-0.793333,-0.304324,-0.909636,-0.427782


##**Merge final DataFrame**

In [42]:
categorical_df = df_copy[['type','title','status','source', 'start_season', 'broadcast_day','real_start_date', 'real_end_date', 'broadcast_time']]
multi_categorical_df = pd.concat([genre_bool, themes_bool], axis=1)
final_df = pd.concat([categorical_df, multi_categorical_df, numerical_df_scaled], axis=1)
final_df = final_df.dropna()
final_df


,type,title,status,source,start_season,broadcast_day,real_start_date,real_end_date,broadcast_time,Action,...,Crossdressing,Reverse Harem,Strategy Game,Childcare,Idols (Male),score,scored_by,episodes,members,favorites
2,tv,Shingeki no Kyojin Season 3 Part 2,finished_airing,manga,spring,monday,2019-04-29,2019-07-01,00:10:00,1.0,...,0.0,0.0,0.0,0.0,0.0,2.393430,6.829660,0.629046,6.052196,9.297283
3,tv,Steins;Gate,finished_airing,visual_novel,spring,wednesday,2011-04-06,2011-09-14,02:05:00,0.0,...,0.0,0.0,0.0,0.0,0.0,2.528907,3.069947,0.123471,2.924724,4.014620
10,tv,Death Note,finished_airing,manga,fall,wednesday,2006-10-04,2007-06-27,00:56:00,0.0,...,0.0,0.0,0.0,0.0,0.0,2.438589,1.780131,0.162361,1.570534,0.495484
11,tv,Jujutsu Kaisen (TV),finished_airing,manga,fall,saturday,2020-10-03,2021-03-27,01:25:00,1.0,...,0.0,0.0,0.0,0.0,0.0,2.416010,1.745532,0.240142,1.985610,4.049381
12,tv,Shigatsu wa Kimi no Uso,finished_airing,manga,fall,friday,2014-10-10,2015-03-20,01:20:00,0.0,...,0.0,0.0,0.0,0.0,0.0,2.009578,2.540278,0.162361,2.516890,3.711050
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
592,tv,Akame ga Kill!,finished_airing,manga,summer,monday,2014-07-07,2014-12-15,00:00:00,1.0,...,0.0,0.0,0.0,0.0,0.0,-1.874101,0.212672,-0.343214,0.245052,-0.324242
599,tv,Owari no Seraph,finished_airing,manga,spring,saturday,2015-04-04,2015-06-20,22:00:00,1.0,...,0.0,0.0,0.0,0.0,0.0,-1.151556,-0.737991,-0.343214,-0.802475,-0.395990
600,tv,Shoujo Kakumei Utena,finished_airing,original,spring,wednesday,1997-04-02,1997-12-24,18:00:00,0.0,...,0.0,0.0,0.0,0.0,0.0,-1.580567,-0.619899,-0.304324,-0.696651,-0.419123
601,tv,Nejimaki Seirei Senki: Tenkyou no Alderamin,finished_airing,light_novel,summer,saturday,2016-07-09,2016-10-01,01:05:00,1.0,...,0.0,0.0,0.0,0.0,0.0,-0.835443,-0.783722,-0.304324,-0.908865,-0.431679


##**FAMD**

In [45]:
famd = prince.FAMD(n_components=10).fit(final_df.dropna())
famd.row_coordinates(final_df)
explained_inertia = famd.explained_inertia_
explained_inertia
explained_inertia_showed = [i*100 for i in explained_inertia]
explained_inertia_showed

[1.821358061063128,
 1.6427656762918166,
 1.540789549509844,
 1.4326727895989153,
 1.322251565629076,
 1.2450722082907044,
 1.1973917950323323,
 1.1526105426639084,
 1.1168507248247048,
 1.097492672329579]